# 멀티모달 RAG 실습
- 모든 모달리티를 하나의 기본 모달리티로 표현하는 방식을 사용해 실습합니다.
### tesseract 설치
  - PDF파일에서 문자열을 추출하는 데 필요한 광학 문자 인식(OCR) 라이브러리 입니다. 
#### Windows 
- tesseract github에서 내려 받을 수 있습니다.  
- https://github.com/UB-Mannheim/tesseract/wiki
- 설치 중에 Additional script data( Hangul script와 Hangul vertical script 체크)
- Additional language data(download) 에서 korean 체크
- 시스템 변수 : C:\Program Files\Tesseract-OCR 추가
- CMD창에서 tesseract --version 입력
#### Mac 
```shell
brew install tesseract
```
#### Linux : 
```shell
sudo apt install tesseract-ocr
sudo apt install libtesseract-dev
```

### poppler 설치
- poppler은 PDF 렌더링과 처리에 필요한 라이브러리입니다.
#### Windows
- poppler github에서 내려받을수 있습니다.
- 압축을 풀고
- 예) C:\utils\poppler-24.08.0\Library\bin를 환경변수로 등록
- 환경변수로 등록하고 powershell에서 pdftoppm -h 명령어가 올바르게 동작한다면 성공적으로 설치한 것입니다.
- https://github.com/oschwartz10612/poppler-windows/releases
### Mac
```shell
brew install poppler
```
### Linux
```shell
sudo apt-get install poppler-utils
```

### unstructured 설치
- tesseract와 poppler를 이용하여 PDF의 전체적인 처리를 수행하는 라이브러리인 unstructured를 설치
```shell
uv add unstructured[all-docs]
```

## 구글 코랩에서 설치
- 구글 코랩은 우분트 운영체제 위에서 동작하도록 구성되어 있습니다.
- Linux 가이드를 따라하면서 설치할 수 있습니다.
```shell
!sudo apt install tesseract-ocr
!sudo apt install labtesseract-dev
!sudo apt-get install poppler-utils
!pip install -U "unstructured[all-docs]"
```

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 데이터 전처리
- 검색에 활용할 PDF 파일의 전처리를 진행하여 벡터 저장소에 저장하는 작업을 진행합니다.
- PDF 내에는 텍스트와 이미지, 테이블 데이터등 다양한 유형의 데이터가 존재합니다.
- 이를 모두 한번에 추출한 뒤, 분류하여 저장하는 작업을 진행합니다.

### unstructured 라이브러리
- 텍스트 전처리 작업에 nltk를 사용하므로, 이에 필요한 데이터를 다음과 같이 다운로드 합니다.

In [15]:
import nltk

# nltk 필요 데이터 다운로드
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\bbang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\bbang\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

- 이어서 partition_pdf 함수를 사용하여 요소 추출을 진행합니다.

In [16]:
from unstructured.partition.pdf import partition_pdf

# PDF에서 요소 추출
raw_pdf_elements = partition_pdf(
    filename="data/sample.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    extract_image_block_output_dir="data"
)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


- filename: 분석하고자 하는 PDF 파일 경로를 지정합니다.
- extract_images_in_pdf: PDF안에 이미지가 있다면 이미지를 추출합니다.
- infer_table_structure: PDF안에 테이블이 있다면 테이블을 추출합니다.
- chunking_strategy: 텍스트를 조각낼 전략을 선택합니다. basic은 섹션 구분 없이 글자 수에 따라 조각 내고, by_title은 페이지 또는 섹션 경계에 따라 조각냅니다.
- extract_image_block_output_dir: 추출한 이미지를 저장할 경로를 설정합니다.

- https://docs.unstructured.io/open-source/core-functionality/chunking

- 다음은 이렇게 추출한 요소들 중, 테이블과 텍스트를 분리하여 저장하는 코드입니다.
- unstructured 라이브러리에서 추출된 데이터의 유형을 검사해 이미지와 테이블 구조를 추출합니다.
- 추출된 element의 type 문자열을 검사했을 때 unstructured.documents.elements.Table이 포함되어 있으면 테이블
- unstructured.documents.elements.CompositeElement가 포함되어 있으면 텍스트 입니다.

In [17]:
# 텍스ㅌ, 테이블 추출
tables = []
texts = []

for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(element) # 테이블 요소 추가
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(element) # 텍스트 요소 추가

In [29]:
tables[0].text

'202314 (1. 1.~12. 31.) A CHHI(4278) 20234 (1. 1.~10. 21.) (1. 202414 1.~10. 19.) Ail 673 (100) 663 (100.0) 630 (100.0) 5.0% ae tt 569 (84.5) 560 (84.5) 526 (83.5) 16.1% OfAt 104 (15.5) 108 (15.5) 104 (16.5) 1.0% we 0-94] 5 (0.7) 5 (0.8) 2( 0.3) 460.0% 10-194 31 ( 4.6) 30 ( 4.5) 20 ( 3.2) A33.3% 20-29M 201 (29.9) 200 (30.2) 209 (33.2) 4.5% 30-39M| 111 (16.5) 110 (16.6) 90 (14.3) A18.2% 40-494 107 (15.9) 104 (15.7) 96 (15.2) AT.7% 50-59M| 60-69M| 63 ( 9.4) 62 ( 9.4) 73 (11.6) 17.7%'

In [24]:
texts[1].text

'1주~42주 665명 630명 35명 말라리아 환자 발생 현황 ’24년 1주부터 42주까지 말라리아 환자는 총 665명(인구 10만 명당 발생률 1.3명)이며, 42주에 10명 신규 발생함 전체 665명 중 국내발생 630명(94.7%), 해외유입 35명(5.3%)으로 해외유입 국가는 주로 아프리카 대륙에 속함 전년 726명 대비 61명(8.4%) 감소 국내 발생 현황 (국내 총 630명) (성별) 남자 526명(83.5%), 여자 104명(16.5%) (연령) 전체 평균 연령 40.7세(범위 2~97세)이며, 20대 209명(33.2%)으로 가장 많았고, 50대 99명(15.7%), 40대 96명(15.2%), 30대 90명(14.3%), 60대 73명(11.6%) 순으로 발생 (신분) 민간인 479명(76.0%), 현역군인 85명(13.5%), 제대군인 66명(10.5%) 순으로 발생 (지역) 경기 356명(56.5%), 인천 120명(19.0%), 서울 84명(13.3%),'

### 멀티 벡터 검색기
- 추출한 요소들을 벡터화하여 저장하고, 이를 통해 멀티모달 데이터를 검색할 수 있는 멀티-벡터 검색기를 구성해보겠습니다.

### 텍스트 및 테이블 요약
- 추출한 요소들은 목잡한 테이블과 텍스트를 그대로 읽은 데이터이기 때문에 복잡하고 가시성 떨어지는 모양을 띠고 있습니다.
- 이를 LLM에 제공하여 텍스트와 테이블에 대한 요약문을 생성하겠습니다.

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# 프롬프트 설정
prompt_text = """당신은 표와 텍스트를 요약하여 검색할 수 있도록 돕는 역할을 맡은 어시스턴트입니다.
이 요약은 임베딩되어 원본 텍스트나 표 요소를 검색하는 데 사용될 것입니다.
표 또는 텍스트에 대한 간결한 요약을 제공하여 검색에 최적화된 형태로 만들어 주세요.
표 또는 텍스트 {element}"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# 텍스트 요약 체인
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# 제공된 텍스트에 대해 요약을 할 경우
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
# 요약을 원치 않을 경우
# text_summaries = texts

# 제공된 테이블에 적용
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

In [35]:
table_summaries[0]

'### 요약\n\n**표 제목:** 연도별 인구 통계 (202314, 20234, 202414)\n\n- **총 인구 수:**\n  - 202314: 673 (100%)\n  - 20234: 663 (100%)\n  - 202414: 630 (100%)\n  \n- **연령대별 분포:**\n  - 0-9세: \n    - 202314: 5 (0.7%)\n    - 20234: 5 (0.8%)\n    - 202414: 2 (0.3%) - 증가율: 460.0%\n  - 10-19세: \n    - 202314: 31 (4.6%)\n    - 20234: 30 (4.5%)\n    - 202414: 20 (3.2%) - 증가율: 33.3%\n  - 20-29세: \n    - 202314: 201 (29.9%)\n    - 20234: 200 (30.2%)\n    - 202414: 209 (33.2%) - 증가율: 4.5%\n  - 30-39세: \n    - 202314: 111 (16.5%)\n    - 20234: 110 (16.6%)\n    - 202414: 90 (14.3%) - 증가율: 18.2%\n  - 40-49세: \n    - 202314: 107 (15.9%)\n    - 20234: 104 (15.7%)\n    - 202414: 96 (15.2%) - 증가율: 11.7%\n  - 50-59세: \n    - 202314: 63 (9.4%)\n    - 20234: 62 (9.4%)\n    - 202414: 73 (11.6%) - 증가율: 17.7%\n\n이 요약은 연도별 인구 통계와 연령대별 분포를 간결하게 정리하여 검색에 최적화된 형태로 제공됩니다.'

In [32]:
text_summaries[0]

'**42주차 (10.13.~10.19.) 요약:**\n\n- **Suyztay**: 국내 발생 및 해외 유입 데이터 포함\n- **aygises zt**: 관련 지표\n- **Bail**: 전체 통계\n\n이 표는 42주차의 국내 발생 및 해외 유입 현황을 요약한 것입니다.'

### 이미지 요약
- 오픈AI에서 지원하는 많은 모델 중 비전, 즉 이미지를 다룰 수 있는 모델은 gpt-4o 모델입니다.

#### 오픈AI의 MLLM
- 오픈 AI의 모든 LLM이 이미지 데이터를 처리할 수 있는 것은 아닙니다.
- 오픈 AI에서는 다음과 같이 이미지를 전달하는 여러 옵션을 제공하고 있습니다.

- 이미지 URL을 통해 모델에게 이미지 전달하기
```python
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content":[
                {"type": "text", "text": "What's in thie image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://~"
                    }                    
                }
            ]
        }
    ],
    max_tokens=300,
)
```

- 이미지를 base64 인코딩하여 모델에게 전달하기
```python
payload = {
    "model": "gpt-4o-mini",
    "messages" :[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64, {base64_image}"
                    }
                }
            ]
        }
    ],
    "max_tokens": 300,
}
resoinse = request.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
```

- 모델에게 여러장의 이미지 전달하기
```python
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content":[
                {
                    "type": "text",
                    "text": "What are in these images? Is there any difference between them?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://~"
                    }
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://~"
                    }
                }
            ]
        }
    ],
    max_tokens=300,
)
```

- 모델에게 이미지 퀄리티 지시하기
```python
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content":[
                {"type": "text", "text": "What's in thie image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://~",
                        "detail": "high"
                    }                    
                }
            ]
        }
    ],
    max_tokens=300,
)
```


## 저해상도 또는 고해상도 이미지
- detail 파라미터를 조정하여 모델이 이미지를 처리하고 텍스트로 이해하는 방식을 제어할 수 있습니다.
- 기본적으로 모델은 auto설정을 사용하여 입력 이미지 크기를 확인한 후, low 또는 high모드를 자동으로 선택합니다.
- low: "저해상도" 모드를 활성화합니다. 이 모드에서는 모델이 512x512크기의 저해상도 이미지를 받고 이미지에 85토큰의 예산을 사용하여 표현합니다. 빠른 응답을 원하거나 높은 세부사항이 필요하지 않은 경우에 유용하며, 입력 토큰수를 절약할 수 있습니다.
- high: "고해상도" 모드를 활성화하며, 먼저 85토큰으로 저해상도 이미지를 확인한후 512x512타일당 170토큰을 사용하여 상세한 크롭을 생성합니다.
- auto: "자동" 모드에서는 입력 이미지 크기에 따라 low 또는 high 설정을 자동으로 선택하여 처리 속도와 이미지 세부 사항을 균형 있게 관리합니다.